# Web Scraping de Palas de Pádel  

- Este script extrae información de palas desde **Padel Nuestro** usando `requests` y `BeautifulSoup`.  
- Primero, obtiene el HTML de cada página de productos y extrae **nombre, precio, atributos y descripción**.  
- Para cada pala, accede a su página individual para recopilar más detalles.  
- Recorre hasta **35 páginas**, guardando los datos en un archivo `palas.json`.  
- Incluye un `time.sleep(1)` para evitar bloqueos por exceso de solicitudes.  
- Se recomienda revisar los **términos de uso** del sitio web antes de hacer scraping. 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

# URL base de las páginas de palas
BASE_URL = "https://www.padelnuestro.com/int/padel-rackets?p={}"

# Encabezados para la solicitud HTTP
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Función para obtener el HTML de la página
def get_html(url):
    response = requests.get(url, headers=HEADERS, verify=False)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Error al acceder a la página: {response.status_code}")

# Función para extraer los detalles de cada pala
def extract_pala_details(soup):
    palas = []
    pala_items = soup.select("li.item.product.product-item")

    for item in pala_items:
        # Nombre y precio de la pala
        name = item.select_one("strong.product-item-name a").text.strip()
        price = item.select_one("span.price-wrapper").text.strip()
        details_url = item.select_one("strong.product-item-name a")["href"]

        # Detalles adicionales desde la página de la pala
        details_html = get_html(details_url)
        details_soup = BeautifulSoup(details_html, "html.parser")

        # Atributos de la pala
        attributes_div = details_soup.select_one("div#product\\.attributes\\.tab")
        attributes = {}
        if attributes_div:
            for attr in attributes_div.select("div.description-attributes"):
                label = attr.select_one("span.description-attributes-label").text.strip()
                value = attr.select_one("span.description-attributes-value").text.strip()
                attributes[label] = value

        # Descripción de la pala
        description_div = details_soup.select_one("div.product.attribute.description div.value")
        description = description_div.text.strip() if description_div else ""

        # Guardar información de la pala
        palas.append({
            "name": name,
            "price": price,
            "attributes": attributes,
            "description": description,
        })

    return palas

# Función principal para recorrer todas las páginas
def main():
    all_palas = []
    try:
        for page in range(1, 36):  # Iterar sobre las 40 páginas
            print(f"Procesando página {page}...")
            url = BASE_URL.format(page)
            html = get_html(url)
            soup = BeautifulSoup(html, "html.parser")
            palas = extract_pala_details(soup)
            all_palas.extend(palas)
            time.sleep(1)  # Evitar ser bloqueado por demasiadas solicitudes

        # Guardar los datos en un archivo JSON
        with open("data/palas.json", "w", encoding="utf-8") as f:
            json.dump(all_palas, f, ensure_ascii=False, indent=4)

        print("Datos guardados en palas.json")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

In [ ]:
import json

# Nombre del archivo donde se guardaron los datos
FILENAME = "palas.json"

# Leer y mostrar los datos del archivo JSON
def print_palas():
    try:
        with open(FILENAME, "r", encoding="utf-8") as f:
            palas = json.load(f)
        
        # Imprimir los datos de cada pala
        for i, pala in enumerate(palas, 1):
            print(f"Pala {i}:")
            print(f"  Nombre: {pala['name']}")
            print(f"  Precio: {pala['price']}")
            print(f"  Descripción: {pala['description']}\n")
            print("  Atributos:")
            for attr, value in pala["attributes"].items():
                print(f"    - {attr}: {value}")
            print("\n" + "-" * 50 + "\n")
    
    except FileNotFoundError:
        print(f"El archivo '{FILENAME}' no existe.")
    except json.JSONDecodeError as e:
        print(f"Error al leer el archivo JSON: {e}")

# Ejecutar la función
print_palas()


# Web Scraping de Palas en Keepadel  

- Este script extrae información de palas de pádel desde **Keepadel** usando `requests` y `BeautifulSoup`.  
- Obtiene el HTML de la página de productos y extrae **nombre, precio, enlace y descripciones**.  
- Para cada pala, accede a su página individual y obtiene una **descripción detallada**.  
- Recorre la página principal y guarda la información en `keepadel_palas.json`.  
- Incluye un `time.sleep(1)` para evitar bloqueos por exceso de solicitudes.  
- Imprime los datos recopilados en la consola antes de guardarlos en JSON.  
- Se recomienda respetar los **términos de uso** del sitio web al hacer scraping.

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

# URL base de las páginas de palas
BASE_URL = "https://keepadel.com/es/9-palas-de-padel?resultsPerPage=99999"

# Encabezados para la solicitud HTTP
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Función para obtener el HTML de la página
def get_html(url):
    response = requests.get(url, headers=HEADERS, verify=False)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Error al acceder a la página: {response.status_code}")

# Función para extraer los detalles de cada pala
def extract_pala_details(soup):
    palas = []
    pala_items = soup.select("div.js-product-miniature-wrapper")

    for item in pala_items:
        # Nombre, precio y enlace de la pala
        name_element = item.select_one("h2.product-title a")
        name = name_element.text.strip() if name_element else "Nombre no disponible"

        price_element = item.select_one("div.product-price-and-shipping span.product-price")
        price = price_element["content"] if price_element and price_element.has_attr("content") else "Precio no disponible"

        details_url = name_element['href'] if name_element else "URL no disponible"

        # Descripción breve desde la página principal
        short_description_element = item.select_one("div.product-description-short a")
        short_description = short_description_element.text.strip() if short_description_element else "Descripción breve no disponible"

        # Detalles adicionales desde la página de la pala
        full_description = "Descripción completa no disponible"
        if details_url != "URL no disponible":
            try:
                details_html = get_html(details_url)
                details_soup = BeautifulSoup(details_html, "html.parser")

                # Descripción detallada de la pala
                description_div = details_soup.select_one("#productdaas-accordion-description .rte-content")
                if description_div:
                    full_description = description_div.get_text(strip=True)
            except Exception as e:
                print(f"Error al procesar {details_url}: {e}")

        # Guardar información de la pala
        palas.append({
            "name": name,
            "price": price,
            "link": details_url,
            "short_description": short_description,
            "full_description": full_description,
        })

    return palas

# Función principal para recorrer todas las páginas
def main():
    all_palas = []
    try:
        for page in range(1, 2):  # Cambiar el rango según el número de páginas
            print(f"Procesando página {page}...")
            url = BASE_URL.format(page)
            html = get_html(url)
            soup = BeautifulSoup(html, "html.parser")
            palas = extract_pala_details(soup)
            all_palas.extend(palas)
            time.sleep(1)  # Evitar ser bloqueado por demasiadas solicitudes

        # Imprimir los datos recopilados
        for idx, pala in enumerate(all_palas, start=1):
            print(f"Pala {idx}:")
            print(f"  Nombre: {pala['name']}")
            print(f"  Precio: {pala['price']}")
            print(f"  Enlace: {pala['link']}")
            print(f"  Descripción breve: {pala['short_description']}")
            print(f"  Descripción completa: {pala['full_description']}\n")

        # Guardar los datos en un archivo JSON
        with open("data/keepadel_palas.json", "w", encoding="utf-8") as f:
            json.dump(all_palas, f, ensure_ascii=False, indent=4)

        print("Datos guardados en keepadel_palas.json")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()



## Padel Mania

- Este script extrae información de palas de pádel desde **Padelmania** usando `requests` y `BeautifulSoup`.  
- Obtiene el HTML de cada página y extrae **nombre, precio, marca, enlace y descripciones**.  
- Para cada pala, accede a su página individual y obtiene una **descripción completa y detalles del producto**.  
- Recorre hasta **72 páginas**, recopilando toda la información disponible.  
- Incluye un `time.sleep(1)` para evitar bloqueos por exceso de solicitudes.  
- Imprime los datos recopilados en la consola y los guarda en `padelmania_palas.json`.  
- Se recomienda respetar los **términos de uso** del sitio web al hacer scraping. 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time

# URL base de las páginas de palas
BASE_URL = "https://padelmania.com/es/5-palas-de-padel?resultsPerPage=99999&page={}"

# Encabezados para la solicitud HTTP
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Función para obtener el HTML de la página
def get_html(url):
    response = requests.get(url, headers=HEADERS, verify=False)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Error al acceder a la página: {response.status_code}")

# Función para extraer los detalles de cada pala
def extract_pala_details(soup):
    palas = []
    pala_items = soup.select("div.js-product-miniature-wrapper")

    for item in pala_items:
        # Nombre, precio y enlace de la pala
        name_element = item.select_one("h2.product-title a")
        name = name_element.text.strip() if name_element else "Nombre no disponible"

        price_element = item.select_one("div.product-price-and-shipping span.product-price")
        price = price_element["content"] if price_element and price_element.has_attr("content") else "Precio no disponible"

        details_url = name_element['href'] if name_element else "URL no disponible"

        # Marca de la pala
        brand_element = item.select_one("div.product-brand a")
        brand = brand_element.text.strip() if brand_element else "Marca no disponible"

        # Descripción breve desde la página principal
        short_description_element = item.select_one("div.product-description-short a")
        short_description = short_description_element.text.strip() if short_description_element else "Descripción breve no disponible"

        # Detalles adicionales desde la página de la pala
        details_html = get_html(details_url) if details_url != "URL no disponible" else ""
        details_soup = BeautifulSoup(details_html, "html.parser") if details_html else None

        # Descripción completa desde el div específico
        description_div = details_soup.select_one("#productdaas-accordion-description .rte-content") if details_soup else None
        full_description = description_div.text.strip() if description_div else "Descripción completa no disponible"

        # Detalles adicionales del producto
        details_div = details_soup.select_one("#productdaas-accordion-details") if details_soup else None
        product_details = details_div.text.strip() if details_div else "Detalles del producto no disponibles"

        # Guardar información de la pala
        palas.append({
            "name": name,
            "price": price,
            "link": details_url,
            "brand": brand,
            "short_description": short_description,
            "full_description": full_description,
            "product_details": product_details
        })

    return palas

# Función principal para recorrer todas las páginas
def main():
    all_palas = []
    try:
        for page in range(1, 72):  # Cambiar el rango según el número de páginas
            print(f"Procesando página {page}...")
            url = BASE_URL.format(page)
            html = get_html(url)
            soup = BeautifulSoup(html, "html.parser")
            palas = extract_pala_details(soup)
            all_palas.extend(palas)
            time.sleep(1)  # Evitar ser bloqueado por demasiadas solicitudes

        # Imprimir los datos recopilados
        for idx, pala in enumerate(all_palas, start=1):
            print(f"Pala {idx}:")
            print(f"  Nombre: {pala['name']}")
            print(f"  Precio: {pala['price']}")
            print(f"  Enlace: {pala['link']}")
            print(f"  Marca: {pala['brand']}")
            print(f"  Descripción breve: {pala['short_description']}")
            print(f"  Descripción completa: {pala['full_description']}")
            print(f"  Detalles del producto: {pala['product_details']}\n")

        # Guardar los datos en un archivo JSON
        with open("data/padelmania_palas.json", "w", encoding="utf-8") as f:
            json.dump(all_palas, f, ensure_ascii=False, indent=4)

        print("Datos guardados en padelmania_palas.json")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


# Unificación de Datos de Palas de Pádel  

- Este script carga datos de palas desde **tres fuentes diferentes** (`Padel Nuestro`, `Keepadel` y `Padelmania`).  
- Usa `json.load()` para leer los archivos `palas.json`, `keepadel_palas.json` y `padelmania_palas.json`.  
- Extrae y normaliza los datos, incluyendo **nombre, precio, atributos, descripción y fuente** de cada pala.  
- Asegura que los atributos opcionales no queden vacíos, manejando posibles errores de codificación o JSON.  
- Genera un archivo final `palas_unificadas.json` con **toda la información consolidada**.  
- Si hay errores al procesar los archivos, los muestra en consola para su depuración.  
- Se recomienda verificar la estructura de los JSON antes de ejecutar el script.  

In [ ]:
import json

def unificar_datos():
    try:
        with open('data/palas.json', encoding="utf-8") as f1, \
             open('data/keepadel_palas.json', encoding="utf-8") as f2, \
             open('data/padelmania_palas.json', encoding="utf-8") as f3:
            
            palas1 = json.load(f1)
            palas2 = json.load(f2)
            palas3 = json.load(f3)

            palas = []

            # Procesar cada JSON
            for pala in palas1:
                palas.append({
                    "name": pala["name"],
                    "price": pala["price"],
                    "attributes": pala.get("attributes", None),  # Evitar valores vacíos innecesarios
                    "description": pala["description"],
                    "source": "https://www.padelnuestro.com/int/padel-rackets?"
                })

            for pala in palas2:
                palas.append({
                    "name": pala["name"],
                    "price": pala["price"],
                    "attributes": None,  # No tiene atributos
                    "description": pala["full_description"],
                    "source": "https://keepadel.com/es/9-palas-de-padel"
                })

            for pala in palas3:
                palas.append({
                    "name": pala["name"],
                    "price": pala["price"],
                    "attributes": pala.get("product_details", None),  # Evita valores vacíos
                    "description": pala["full_description"],
                    "source": "https://padelmania.com/es/5-palas-de-padel"
                })

            return palas

    except UnicodeDecodeError as e:
        print(f"Error de codificación: {e}")
    except json.JSONDecodeError as e:
        print(f"Error al cargar JSON: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")
    return []

# Ejecutar función y guardar los datos
palas = unificar_datos()

if palas:
    with open('data/palas_unificadas.json', 'w', encoding="utf-8") as f:
        json.dump(palas, f, indent=4, ensure_ascii=False)  # `ensure_ascii=False` para caracteres especiales
    print(f"Se han guardado {len(palas)} palas en 'data/palas_unificadas.json'")
else:
    print("No se han podido unificar los datos.")


# Codigo IA Palas

In [ ]:
import json
import openai
from docx import Document
import numpy as np

# Generación y Búsqueda de Embeddings con Azure OpenAI  

- Este script procesa **textos largos** dividiéndolos en *chunks* con solapamiento para mejorar la segmentación.  
- Utiliza `OpenAIEmbeddings` de **Azure OpenAI** para convertir los *chunks* en vectores de embeddings.  
- Los embeddings se generan con el modelo `"text-embedding-ada-002"`, utilizando la API de Azure OpenAI.  
- Para buscar información relevante, calcula la **similitud del coseno** entre la consulta y los *chunks* almacenados.  
- Ordena los resultados por relevancia y devuelve los **top-k segmentos** más similares a la consulta.  
- Incluye un ejemplo práctico con un texto de prueba y una consulta sobre palas de pádel.  
- Es necesario configurar correctamente las credenciales (`API_KEY`, `API_BASE`, `API_VERSION`) antes de ejecutar.  


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
import numpy as np


# Función para dividir el texto en chunks
def split_text_into_chunks(text, chunk_size=1000, overlap=200):
    """
    Divide un texto largo en chunks de tamaño específico con un solapamiento opcional.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

# Función para generar embeddings usando Azure OpenAI
def generate_embeddings_azure(chunks, api_key, api_base, api_version, deployment):
    """
    Genera embeddings para una lista de chunks de texto utilizando Azure OpenAI.
    """
    embeddings = OpenAIEmbeddings(
        deployment=deployment,
        model="text-embedding-ada-002",
        openai_api_key=api_key,
        openai_api_base=api_base,
        openai_api_version=api_version
    )
    chunk_vectors = []
    for chunk in chunks:
        vector = embeddings.embed_query(chunk)
        chunk_vectors.append({"chunk": chunk, "vector": vector})
    return chunk_vectors

# Función para buscar información relevante manualmente
def search_relevant_chunks(query, chunk_vectors, api_key, api_base, api_version, deployment, top_k=5):
    """
    Busca los chunks más relevantes para una consulta utilizando la similitud de coseno.
    """
    embeddings = OpenAIEmbeddings(
        deployment=deployment,
        model="text-embedding-ada-002",
        openai_api_key=AZURE_OPENAI_API_KEY,
        openai_api_base=AZURE_OPENAI_ENDPOINT,
        openai_api_version=OPENAI_API_VERSION
    )
    query_vector = embeddings.embed_query(query)
    
    # Calcular similitudes por coseno
    similarities = []
    for chunk_data in chunk_vectors:
        chunk_vector = chunk_data["vector"]
        similarity = np.dot(query_vector, chunk_vector) / (np.linalg.norm(query_vector) * np.linalg.norm(chunk_vector))
        similarities.append((chunk_data["chunk"], similarity))
    
    # Ordenar por relevancia
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

# Ejemplo de uso
if __name__ == "__main__":
    # Texto de ejemplo
    documentation_text = "Este es un texto de ejemplo que representa la documentación para buscar la pala ideal." * 20

    # Dividir en chunks
    chunks = split_text_into_chunks(documentation_text)

    AZURE_OPENAI_API_KEY = "AZURE_OPENAI_API_KEY"
    AZURE_OPENAI_ENDPOINT = "AZURE_OPENAI_ENDPOINT"
    OPENAI_API_VERSION = "2023-05-15"
    # Configuración de Azure OpenAI
    
    EMBEDDING_DEPLOYMENT = "text-embedding-ada-002"

    # Generar embeddings para los chunks
    chunk_vectors = generate_embeddings_azure(chunks, AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, OPENAI_API_VERSION, EMBEDDING_DEPLOYMENT)
    print("Embeddings generados para los chunks.")

    # Consulta del usuario
    query = "Soy un jugador intermedio que busca una pala con control y potencia moderada."

    # Buscar chunks relevantes
    top_chunks = search_relevant_chunks(query, chunk_vectors, AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT, OPENAI_API_VERSION, EMBEDDING_DEPLOYMENT)
    print("\nChunks más relevantes:")
    for i, (chunk, similarity) in enumerate(top_chunks):
        print(f"{i + 1}. Similitud: {similarity:.4f}")
        print(f"Chunk: {chunk}")


# Asistente de Pádel con Azure OpenAI  

- Este script utiliza **Azure OpenAI** para recomendar una pala de pádel basada en características del jugador.  
- Carga un documento Word con información sobre la elección de palas y lo usa como referencia.  
- El asistente considera **nivel de juego, estilo de juego y lesiones** para hacer recomendaciones personalizadas.  
- Genera una respuesta en **formato JSON**, incluyendo aspectos clave como **forma, peso, balance y materiales**.  
- Utiliza `gpt-35-turbo` para procesar la consulta del jugador y devolver una recomendación óptima.  
- Es necesario configurar correctamente las credenciales (`API_KEY`, `API_BASE`, `API_VERSION`) antes de ejecutar.  

In [ ]:
import os
from docx import Document
from openai import AzureOpenAI
import json

# Configuración de Azure OpenAI
AZURE_OPENAI_API_KEY = "AZURE_OPENAI_API_KEY"
AZURE_OPENAI_ENDPOINT = "AZURE_OPENAI_ENDPOINT"
OPENAI_API_VERSION = "2023-05-15"

client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# Ruta del archivo Word
file_path = r"C:\Users\jgutierrezdecalderon\OneDrive - KPMG\Desktop\Personal\Proyectos\Padel_chatbot\Documentacion_elleccion_pala.docx"

# Leer el contenido del archivo Word
def read_word_file(file_path):
    doc = Document(file_path)
    content = "\n".join([paragraph.text for paragraph in doc.paragraphs if paragraph.text.strip()])
    return content

# Extraer contenido del archivo
document_content = read_word_file(file_path)

# Variables del jugador (estas pueden venir de un formulario o sistema externo)
nivel_juego = 5  # Nivel del jugador: 0 (principiante) a 7 (avanzado/pro)
tipo_jugador = "potencia"  # Puede ser "control", "equilibrado" o "potencia"
lesion_codo = "si"  # Puede ser "si" o "no"

propt_assistant_padel="""Eres un asistente de pádel que ayuda a los jugadores a elegir la pala ideal para su juego. 
Debes tener en cuenta el nivel de juego, el tipo de jugador y si tiene lesión de codo. 
Configura la pala recomendada en base a estos criterios en cuanto a forma, peso, material, grosor, balance y superficie. 
Usa la documentación proporcionada para reforzar tus recomendaciones.
Ten en cuenta también la descripción de la pala que te da el jugador o de su necesidad en la pregunta

El nivel de juego del jugador es {nivel_juego} sobre 7
El tipo de jugador es {tipo_jugador}
Tiene lesión de codo. {lesion_codo}

La respuesta será en formato json poniendo la recomendación de cada componente"""


# Crear el prompt para el asistente
response = client.chat.completions.create(
    model="gpt-35-turbo",
    messages=[
        {"role": "system", "content": propt_assistant_padel },
        {"role": "system", "content": f"Documentación sobre la elección de pala:\n{document_content}"},
        {"role": "user", "content": f"Soy un jugador de dercha que es algo agresivo, me gusta que la pelota coja efectos pero sentirme cómo desde atrás"}
    ]
)

# Recomendaciones generadas por el asistente
recomendaciones = json.loads(response.choices[0].message.content)
print(recomendaciones)

# Generación de embeddings


- Carga un archivo JSON que contiene información sobre palas (palas_unificadas.json).
- Define una ruta de archivo para guardar los embeddings generados (palas_embeddings.json).
- Utiliza la API de OpenAI para generar embeddings a partir de descripciones de palas.
- Verifica si existen embeddings previamente generados y los carga si es necesario.
- Filtra las palas ya procesadas para evitar duplicados en los embeddings.
- Solo genera embeddings para aquellas palas que tienen descripción y no han sido procesadas antes.
- Guarda los embeddings generados en un archivo JSON para su uso posterior.

In [ ]:
import json
import openai
import numpy as np
import os

import json
import os

# Cargar el JSON de las palas
with open("data/palas_unificadas.json", "r", encoding="utf-8") as file:
    palas_data = json.load(file)

# Ruta para guardar los embeddings
EMBEDDINGS_FILE = "palas_embeddings.json"

def generate_embeddings(text, model="text-embedding-ada-002"):  # model = "deployment_name"
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

# Función para generar embeddings para cada pala
def generar_embeddings_palas(palas, embeddings_file):
    # Cargar embeddings existentes si el archivo ya existe
    if os.path.exists(embeddings_file):
        with open(embeddings_file, "r", encoding="utf-8") as ef:
            embeddings = json.load(ef)
    else:
        embeddings = []

    # Crear un set con los IDs de las palas ya procesadas
    procesadas = {item["pala"]["name"] for item in embeddings}

    # Generar embeddings solo para palas nuevas
    for pala in palas:
        if pala["name"] not in procesadas:
            description = pala.get("description", "")
            if description:  # Solo generamos embeddings si hay descripción
                embedding = generate_embeddings(description)
                embeddings.append({"pala": pala, "embedding": embedding})

    # Guardar embeddings actualizados
    with open(embeddings_file, "w", encoding="utf-8") as ef:
        json.dump(embeddings, ef, ensure_ascii=False, indent=4)

    return embeddings

# Generar y guardar embeddings
embeddings_palas = generar_embeddings_palas(palas_data, EMBEDDINGS_FILE)

In [ ]:
def filtrar_por_precio(embeddings_file, min_price, max_price):
    # Leer el archivo de embeddings
    with open(embeddings_file, "r", encoding="utf-8") as ef:
        embeddings = json.load(ef)

    # Filtrar las palas por rango de precios
    palas_filtradas = []
    for item in embeddings:
        try:
            precio = float(item["pala"]["price"].strip("€").replace(",", "."))
            if min_price <= precio <= max_price:
                palas_filtradas.append(item)
        except ValueError:
            continue  # Si no se puede procesar el precio, ignorar la pala

    return palas_filtradas

# Definir rango de precios
min_price = 50  # Precio mínimo
max_price = 200  # Precio máximo

# Filtrar palas por precio utilizando el archivo de embeddings
embeddings_palas_filtradas = filtrar_por_precio(EMBEDDINGS_FILE, min_price, max_price)

# Búsqueda por similitud

- cosine_similarity(a, b): Calcula la similitud coseno entre dos vectores a y b para medir su semejanza. Se utiliza la fórmula estándar del coseno de los ángulos entre los vectores.

- get_embedding(text, model): Genera un embedding para un texto proporcionado utilizando el modelo de OpenAI (por defecto, text-embedding-ada-002). Los embeddings son representaciones numéricas del texto.

- buscar_palas_por_similitud(consulta, embeddings_palas, top_n): Realiza una búsqueda de las palas más similares a una consulta de texto:

- Genera un embedding para la consulta.
Calcula la similitud coseno entre el embedding de la consulta y los embeddings de las palas.
Ordena los resultados por similitud en orden descendente.
Devuelve las top_n palas más similares.
Consulta del usuario: El código simula una consulta con texto (especificando las condiciones para buscar una pala), que se utiliza en la función de búsqueda.

- Mostrar resultados: Imprime los resultados de las palas más similares, mostrando el nombre, la similitud, el precio y la fuente de cada pala.

In [ ]:

import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def get_embedding(text, model="text-embedding-ada-002"):  # model = "deployment_name"
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding


def buscar_palas_por_similitud(consulta, embeddings_palas, top_n=5):
    # Generar embedding para la consulta
    consulta_embedding = get_embedding(consulta, model="text-embedding-ada-002")

    # Calcular similitud entre la consulta y las palas
    resultados = []
    for item in embeddings_palas:
        pala = item["pala"]
        embedding = item["embedding"]
        similitud = cosine_similarity(consulta_embedding, embedding)
        resultados.append({"pala": pala, "similitud": similitud})

    # Ordenar por similitud descendente
    resultados_ordenados = sorted(resultados, key=lambda x: x["similitud"], reverse=True)
    return resultados_ordenados[:top_n]


# Consulta del usuario
consulta = """
Ayudame a buscar la pala que más se ajuste a estas condiciones:
{recomendaciones}
"""

# Realizar búsqueda
top_palas = buscar_palas_por_similitud(consulta, embeddings_palas_filtradas, top_n=10)

# Mostrar resultados
for i, result in enumerate(top_palas, 1):
    print(f"{i}. {result['pala']['name']} - Similitud: {result['similitud']:.2f} - Precio: {result['pala']['price']} - Web {result['pala']['source']}")